# Download RGB

1 - General setting up

In [2]:
# import libraries
import ee
import geemap
ee.Authenticate()
ee.Initialize(project= 'kochias2')
cpus_ = 7



2 - Select a province to download

In [3]:
## MANITOBA
prov = 'Man.'
provName = 'MB'
# asset_path = 'projects/just-amp-296821/assets/road_mb_utm14'# MB
out_dir = r'C:\HA\CSA_N2O\Field_boundary\github\CSA_Field_Boundary_Segmentation\6. Data'


### SASKATCHEWAN
# prov = 'Sask.'
# provName = 'SK'
# asset_path = "projects/just-amp-296821/assets/road_sk_utm13" # SK
# out_dir = r'D:\HA\CSA_N2O\Field_boundary\Raw_RGB_SK'


# ### ABLERTA
# prov = 'Alb.'
# provName = 'AB'
# asset_path = 'projects/ee-aafc-annimation/assets/alberta_highway'# AB
# out_dir = r'C:\HA\CSA_N2O\Field_boundary\github\CSA_Field_Boundary_Segmentation\6. Data'


3 - Get tiles

In [4]:
allProvince = ee.FeatureCollection('projects/ee-aafc-annimation/assets/provincialBoundary')
pruid_list = allProvince.aggregate_array('PRFABBR')
selectProv = allProvince.filter(ee.Filter.stringStartsWith('PRFABBR', prov))
# selectProv

# Load full grid and convert to list
grid = ee.FeatureCollection('projects/ee-download-canada/assets/Grid_prairies').filterBounds(selectProv)
list_roi_all = grid.toList(grid.size())
grid_size = list_roi_all.size().getInfo()
# grid_size

4 - Download RGB

In [5]:
def build_cropland_mask(geom: ee.Geometry) -> ee.Image:
    """
    Builds binary cropland mask: selected AAFC classes → 1, all others → 0.
    Combines AAFC multi-year crop classification with ESA cropland mask.
    """
    TARGET_CLASSES = ee.List([
        132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 145, 146, 147, 148, 149,
        150, 151, 152, 153, 154, 155, 156, 157,
        158, 160, 162, 167, 174
    ])

    # ESA cropland (v100, class 40)
    esa = ee.ImageCollection("ESA/WorldCover/v100").first().clip(geom)
    esa_crop = esa.eq(40)

    # Filter AAFC ACI to time window and remap target classes
    aci = (
        ee.ImageCollection("AAFC/ACI")
        .filterDate("2018-01-01", "2024-12-31")
        .map(lambda img: img.select(0)
            .remap(TARGET_CLASSES, ee.List.repeat(1, TARGET_CLASSES.size()), 0)
            .rename("cropSel")
            .toUint8()
            .clip(geom))
    )

    # Collapse multi-year into one mask (any year with target crop = 1)
    aafc_mask = aci.max().rename("aafcMask")

    # Combine AAFC and ESA cropland masks
    mask = (
        ee.Image(aafc_mask)
        .updateMask(esa_crop)
        .rename("mask")
        .unmask(0)
        .clip(geom.buffer(30))
    )

    return mask
def get_crp_mask(download_dir, tile_id, tile):
    import ee, os
    from pathlib import Path
    import geemap

    # ee.Initialize()

    output_tif = os.path.join(download_dir, f'crop_mask_{tile_id}_1.tif')

    if os.path.exists(output_tif):
        print(f"Crop mask exists for file {Path(output_tif).stem}, skipping...")
    else:
        crop_mask_raster = build_cropland_mask(tile.geometry())
        geemap.download_ee_image_tiles(
            crop_mask_raster,
            ee.FeatureCollection(tile),
            download_dir,
            prefix=f'crop_mask_{tile_id}_',
            crs="EPSG:4326",
            scale=10
        )

def parallelize_download(func, argument_list, num_processes):
    from multiprocessing import Pool
    from tqdm import tqdm
    import time

    pool = Pool(processes=num_processes)
    jobs = [
        pool.apply_async(func=func, args=argument)
        for argument in argument_list
    ]
    pool.close()

    result_list_tqdm = []
    for job in tqdm(jobs):
        result_list_tqdm.append(job.get())
        time.sleep(5)  # Short delay to avoid overloading

    return result_list_tqdm

In [6]:
argument_list_config_all = [(out_dir, str(local_idx), ee.Feature(list_roi_all.get(local_idx)))
    for local_idx in range(grid_size)]


In [ ]:
result_list = parallelize_download(
    func=get_crp_mask,
    argument_list=argument_list_config_all[0:2],
    num_processes=cpus_
)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def export_mask_to_drive(tile_id, tile):
    print(f"Starting export for tile {tile_id}...")

    try:
        mask = build_cropland_mask(tile.geometry())

        task = ee.batch.Export.image.toDrive(
            image=mask,
            description=f"crop_mask_{tile_id}",
            folder="EarthEngineExports",  # You can change this folder name
            fileNamePrefix=f"crop_mask_{tile_id}",
            region=tile.geometry().bounds().getInfo()['coordinates'],
            scale=10,
            crs="EPSG:4326",
            maxPixels=1e13
        )
        task.start()
        print(f"✓ Export task started for tile {tile_id}")
    except Exception as e:
        print(f"❌ Failed to export tile {tile_id}: {e}")

In [ ]:
tile_index = 0  # or any tile you want
tile = ee.Feature(list_roi_all.get(tile_index))
export_mask_to_drive(str(tile_index), tile)